In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gtsrb-german-traffic-sign
/kaggle/input/gtsrb-german-traffic-sign
/kaggle/input/gtsrb-german-traffic-sign
/kaggle/input/gtsrb-german-traffic-sign/Meta
/kaggle/input/gtsrb-german-traffic-sign/Meta
/kaggle/input/gtsrb-german-traffic-sign/Meta
/kaggle/input/gtsrb-german-traffic-sign/Meta
/kaggle/input/gtsrb-german-traffic-sign/Meta
/kaggle/input/gtsrb-german-traffic-sign/Meta
/kaggle/input/gtsrb-german-traffic-sign/Meta
/kaggle/input/gtsrb-german-traffic-sign/Meta
/kaggle/input/gtsrb-german-traffic-sign/Meta
/kaggle/input/gtsrb-german-traffic-sign/Meta
/kaggle/input/gtsrb-german-traffic-sign/Meta
/kaggle/input/gtsrb-german-traffic-sign/Meta
/kaggle/input/gtsrb-german-traffic-sign/Meta
/kaggle/input/gtsrb-german-traffic-sign/Meta
/kaggle/input/gtsrb-german-traffic-sign/Meta
/kaggle/input/gtsrb-german-traffic-sign/Meta
/kaggle/input/gtsrb-german-traffic-sign/Meta
/kaggle/input/gtsrb-german-traffic-sign/Meta
/kaggle/input/gtsrb-german-traffic-sign/Meta
/kaggle/input/gtsrb-germa

### Importing libraries

In [ ]:
# standard libraries
import numpy as np 
import pandas as pd 
import tensorflow as tf
import os

# Image related
import cv2
from PIL import Image

# Moedl evaluation
from sklearn import metrics 
from sklearn.metrics import f1_score, accuracy_score,confusion_matrix,ConfusionMatrixDisplay,classification_report,roc_auc_score,auc
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
%matplotlib inline


# Keras library
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import Sequence
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger

# Importing VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16

# Time
import time
import datetime

In [ ]:
# defining variables
data = []
labels = []
classes = 10
cur_path = os.getcwd()

# Retrieving the images and their labels 
for i in range(classes):
    train_dir = os.path.join('../input/gtsrb-german-traffic-sign/','train')
    # Listing all subdirectories 
    all_classes = os.listdir(train_dir)
    sorted_classes = sorted(all_classes, key=lambda x: int(x))
    
    # Selecting the first 10 classes
    selected_classes = sorted_classes[:10]
    
print(selected_classes)

In [ ]:
# Numebr of images per class
total_images = 0
for class_name in selected_classes:
    class_dir = os.path.join(train_dir, class_name)
    num_images = len(os.listdir(class_dir))
    total_images += num_images

# percentage distribution per class
for class_name in selected_classes:
    class_dir = os.path.join(train_dir, class_name)
    num_images = len(os.listdir(class_dir))
    print(f"Class: {class_name}, Number of Images: {num_images} , Percentage distribution:{np.round(num_images/total_images*100,2)}%")
print(f"Total Images: {total_images}")

In [ ]:
# Loading a subset of data 
max_images_per_class = 1000 # 1000 images per class
images = []
labels = []

for class_id, class_name in enumerate(selected_classes):
    class_dir = os.path.join(train_dir, class_name)
    img_files = os.listdir(class_dir)[:max_images_per_class]
    for img_file in img_files: 
        img_path = os.path.join(class_dir, img_file)  
        try:
            img = Image.open(img_path).resize((224,224)) # input format of VGG
            img = np.array(img)
            if img.shape[2] == 4:  
                img = img[:,:,:3]  # Keep only the first 3 channels (RGB)
            images.append(img)
            labels.append(class_id)
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            continue

X = np.array(images)
y = np.array(labels)

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

In [ ]:
print("Original Class Distribution:", Counter(y))

# Applying Data augmentation on Minority classes to handle class imbalance
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

# Calculating max samples per class
max_samples = max(Counter(y).values())

# Augment images for each class
balanced_images = []
balanced_labels = []

for class_id in range(len(selected_classes)):
    class_images = X[y == class_id]
    class_count = len(class_images)
    
    balanced_images.extend(class_images)
    balanced_labels.extend([class_id] * class_count)
    
    images_to_augment = max_samples - class_count
    if images_to_augment > 0:
        augmented_flow = datagen.flow(class_images, batch_size=1)
        for _ in range(images_to_augment):
            aug_image = next(augmented_flow)[0].astype('uint8')
            balanced_images.append(aug_image)
            balanced_labels.append(class_id)

X_balanced = np.array(balanced_images)
y_balanced = np.array(balanced_labels)

print(f"Balanced X shape: {X_balanced.shape}")
print(f"Balanced y shape: {y_balanced.shape}")
print("Balanced Class Distribution:", Counter(y_balanced))

# Example of a augmented image
plt.figure(figsize=(10, 10))
for i in range(9):
    plt.subplot(3, 3, i + 1)
    aug_img = datagen.random_transform(X_balanced[0])
    plt.imshow(aug_img.astype('uint8'))
    plt.axis('off')
plt.show()

### Preparing test data set

In [ ]:
# Defining test dataset path
test_csv_path = os.path.join('../input/gtsrb-german-traffic-sign/Test.csv')
test_dir = '../input/gtsrb-german-traffic-sign/'

# Load the Test.csv file
test_df = pd.read_csv(test_csv_path)

# Filter for selected class IDs (0-9)
selected_classes = list(range(10)) 
filtered_test_df = test_df[test_df['ClassId'].isin(selected_classes)]

# Initialize lists to store test images and labels
test_images = []
test_labels = []

# Loop through the filtered DataFrame
for _, row in filtered_test_df.iterrows():
    img_path = os.path.join(test_dir,row['Path'])
    class_id = row['ClassId']
    try:
        img = Image.open(img_path).resize((224, 224))  # Resize for VGG16
        img = np.array(img)
        if img.shape[2] == 4:  # Handle RGBA images
            img = img[:, :, :3]
        test_images.append(img)
        test_labels.append(class_id)
    except Exception as e:
        print(f"Error processing image {img_path}: {e}")
        continue

# Convert lists to numpy arrays
X_test = np.array(test_images)
y_test = np.array(test_labels)

# Normalize test images to range [0, 1]
X_test = X_test.astype('float32') / 255.0

# Print shapes to verify
print(f"Test data shape: {X_test.shape}")
print(f"Test labels shape: {y_test.shape}")

In [ ]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y)

# Shape of train and val datasets
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}\n")

print(f"X_val shape: {X_val.shape}")
print(f"y_val shape: {y_val.shape}")

# Normalize the images to [0, 1] range
X_train = X_train.astype('float32') / 255.0
X_val = X_val.astype('float32') / 255.0

In [ ]:
# Define a data generator class to import data
class DataGenerator(Sequence):
    def __init__(self, X, y, batch_size,num_classes):
        self.X = X
        self.y = to_categorical(y, num_classes=num_classes)
        self.batch_size = batch_size
        self.indices = np.arange(len(y))

    def __len__(self):
        return int(np.ceil(len(self.y) / self.batch_size))

    def __getitem__(self, idx):
        batch_indices = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_X = self.X[batch_indices]
        batch_y = self.y[batch_indices]
        return batch_X, batch_y

# Create data generators for training and validation
batch_size = 16  # Reduce batch size to manage memory
train_gen = DataGenerator(X_train, y_train, batch_size,num_classes =10)
val_gen = DataGenerator(X_val, y_val, batch_size,num_classes =10)

## Defining the Deep Neural Network Model

In [ ]:
# Load the pretrained VGG16 model without the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model to retain pretrained weights
base_model.trainable = False

# Add custom layers on top
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)  # Add a dense layer
x = Dropout(0.3)(x)  # Add dropout to reduce overfitting
output = Dense(10, activation='softmax')(x)  # Final layer for 10 classes

# Create the model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
from tensorflow.keras.losses import CategoricalCrossentropy

loss_fn = CategoricalCrossentropy()

model.compile(optimizer=Adam(), loss=loss_fn, metrics=['accuracy'])

model.summary()

#### training the model 

In [ ]:
# Callbacks for tracking and optimization
checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss', mode='min')
early_stopping = EarlyStopping(patience=20, monitor='val_loss', mode='min', restore_best_weights=True)
csv_logger = CSVLogger('training_log.csv')

# Train the model using the generators
history = model.fit(
    train_gen,
    epochs=30,
    validation_data=val_gen,
    callbacks=[checkpoint, early_stopping, csv_logger],
    verbose=1
)

### Model Evaluation on Test Set

In [ ]:
# Load the best model
model.load_weights('best_model.keras')

# Assuming test_gen is the data generator that provides batches
test_gen = DataGenerator(X_test, y_test, batch_size, num_classes=10)

# Predict on the test set (this will give you the probabilities for each class)
test_predictions = model.predict(test_gen)
test_classes = np.argmax(test_predictions, axis=1)

# Test AUC
print(f"Test AUC: {roc_auc_score(y_test, test_predictions, multi_class='ovr'):.4f}")

# Classificatio report
print(classification_report(y_test, test_classes))

#### Visualisation of model performance
* Confusion Matrix
* Precision-Recall Curve

In [ ]:
# Confusion Matrix
plt.figure(figsize=(10, 5))
cm = confusion_matrix(y_test, test_classes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=selected_classes)
disp.plot(cmap='Blues', xticks_rotation='vertical')
plt.title("Confusion Matrix")
plt.show()

# ROC Curve subplot 
print('-'*100)
fpr = {}
tpr = {}
roc_auc = {}

y_test_onehot = np.zeros((len(y_test), len(selected_classes)))
y_test_onehot[np.arange(len(y_test)), y_test] = 1

for i, class_name in enumerate(selected_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_onehot[:, i], test_predictions[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

plt.figure(figsize = (15,4))
plt.subplot(1, 2, 1)
for i, class_name in enumerate(selected_classes):
    plt.plot(fpr[i], tpr[i], label=f"Class {class_name} (AUC = {roc_auc[i]:.2f})")
plt.plot([0, 1], [0, 1], 'k--', lw=2, label="Chance (AUC = 0.50)")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend(loc="lower right")
plt.grid()

# Precision-Recall Curve subplot 
precision = {}
recall = {}
pr_auc = {}

plt.subplot(1, 2, 2)
for i, class_name in enumerate(selected_classes):
    precision[i], recall[i], _ = precision_recall_curve(y_test_onehot[:, i], test_predictions[:, i])
    pr_auc[i] = auc(recall[i], precision[i])

    plt.plot(recall[i], precision[i], label=f"Class {class_name} (AUC = {pr_auc[i]:.2f})")

plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve")
plt.legend(loc="lower left")
plt.grid()

# Show the plots
plt.show()

### Model Error Evalaution

In [ ]:
# misclassified samples
misclassified_indices = np.where(y_test != test_classes)[0]

# Filter misclassified samples for classes 3 and 6
misclassified_3 = [i for i in misclassified_indices if y_test[3]]
misclassified_6 = [i for i in misclassified_indices if y_test[6]]

# Optionally, if you want to examine these samples
misclassified_samples_3 = X_test[misclassified_3]
misclassified_samples_6 = X_test[misclassified_3]

#### Misclassified samples class 3

In [ ]:
# Example of 10 images of class 3 that are misclassifed
for i in range(len(misclassified_3)):
    idx = misclassified_3[i]  
    image = misclassified_samples[i]  
    true_label = y_test[idx]  
    predicted_label = test_classes[idx]  
    if i == 10:
        break
    # Reverse normalization and convert the image back to uint8
    image = image * 255
    image = image.astype('uint8')
    
    plt.imshow(image)
    plt.title(f"True label: {true_label}, Predicted: {predicted_label}")
    plt.show()

#### Misclassified samples class 6

In [ ]:
# Example of 10 images of class 6 that are misclassifed
for i in range(len(misclassified_6)):
    idx = misclassified_3[i]  
    image = misclassified_samples[i]  
    true_label = y_test[idx]  
    predicted_label = test_classes[idx]  
    if i == 10:
        break
    # Reverse normalization and convert the image back to uint8
    image = image * 255
    image = image.astype('uint8')
    
    plt.imshow(image)
    plt.title(f"True label: {true_label}, Predicted: {predicted_label}")
    plt.show()

### Save the model

In [ ]:
model.save('road_sign_classifier.h5')